In [ ]:
## Vae for dna barcode sequence 50 dim and mahalanobis distance

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras.losses import binary_crossentropy
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import io
from google.colab import files
from numpy import argmax
from numpy import array
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from scipy.spatial import distance

original_dim = 3290
intermediate_dim = 256
latent_dim = 50

In [ ]:
uploaded = files.upload()

Saving test_features.csv to test_features.csv
Saving train_features.csv to train_features.csv
Saving train_labels.csv to train_labels.csv


In [ ]:
train_features_df = pd.read_csv('train_features.csv')
train_labels_df = pd.read_csv('train_labels.csv')
test_features_df = pd.read_csv('test_features.csv')

In [ ]:
#get dna barcode sequencing data
def clean_seq(seq_df):
    sequences = seq_df['dna'].str.replace('[^ACGT]','N') #Converting all characters other than A, C, G, T as N
    for i in range(len(sequences)):
        sequences[i] = sequences[i].ljust(1058,'N')
    #print(sequences[0])
    
    sequences_array = np.array(sequences)
    #print(pd.DataFrame(sequences_array))
    seq_list_padded = []
    for i in range(len(sequences_array)):
        list_seq = list(sequences_array[i])
        del list_seq[658:] #truncating seq after 658 chacters
        seq_list_padded.append(list_seq)
    print(len(list_seq))
    replace_map = {'A': 1, 'C': 2, 'G': 3, 'T': 4, 'N': 5} 
    integer_encoded = []
    for i in range(len(seq_list_padded)):
        C = (pd.Series(seq_list_padded[i])).map(replace_map) #convert the list to a pandas series temporarily before mapping
        integer_encoded.append(list(C))
    data = np.array([1, 2, 3, 4, 5])
    data = data.reshape(-1, 1) 
    integer_encoded = np.array(integer_encoded).astype(int)
    onehot_encoder = OneHotEncoder(sparse=False)
    onehot_encoder.fit(data)
    one_hot_encoded_seqs = []
    for i in range(len(integer_encoded)):
        integer_encoded_tmp = integer_encoded[i].reshape(len(integer_encoded[i]), 1)
        onehot_encoded = np.array(onehot_encoder.transform(integer_encoded_tmp)).flatten()
        one_hot_encoded_seqs.append(onehot_encoded)
    one_hot_encoded_seqs_array = np.array(one_hot_encoded_seqs)
    #one_hot_encoded_seqs_array = one_hot_encoded_seqs_array.reshape(len(one_hot_encoded_seqs_array), 60, 60)
    #one_hot_encoded_seqs_array = np.expand_dims(one_hot_encoded_seqs_array, axis=3)
    
    return one_hot_encoded_seqs_array

def load_n_encode_labels(df):
    labels = df['labels']
    softmax_layer = len(set(labels))
    le = LabelEncoder()
    le.fit(labels)
    label_seq = le.transform(labels)
    
    return label_seq, le, softmax_layer

def decode_labels(encoded_predict_labels, le):
    test_predictions = le.inverse_transform(encoded_predict_labels)
    
    return test_predictions

In [ ]:
one_hot_encoded_seqs_array_train = clean_seq(train_features_df)
one_hot_encoded_seqs_array_test = clean_seq(test_features_df)
one_hot_encoded_seqs_array = np.row_stack((one_hot_encoded_seqs_array_train, one_hot_encoded_seqs_array_test)) #combing train and test features
x_train = one_hot_encoded_seqs_array
x_validation = one_hot_encoded_seqs_array_test
train_size = len(x_train)
batch_size = 32


658
658


In [ ]:
label_seq, le, softmax_layer = load_n_encode_labels(train_labels_df) #label_seq is encoded labels

In [ ]:
print(one_hot_encoded_seqs_array_train.shape)
print(one_hot_encoded_seqs_array_test.shape)
print(x_train.shape)
print(x_validation.shape)
x_train = x_train.astype(np.float32)

(12906, 3290)
(8306, 3290)
(21212, 3290)
(8306, 3290)


In [ ]:
# make sampling layer
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
# Define encoder model.
original_inputs = tf.keras.Input(shape=(original_dim,), name="encoder_input")
x = layers.Dense(intermediate_dim, activation="relu")(original_inputs)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()((z_mean, z_log_var))
encoder = tf.keras.Model(inputs=original_inputs, outputs=z, name="encoder")

# Define decoder model.
latent_inputs = tf.keras.Input(shape=(latent_dim,), name="z_sampling")
x = layers.Dense(intermediate_dim, activation="relu")(latent_inputs)
outputs = layers.Dense(original_dim, activation="sigmoid")(x)
decoder = tf.keras.Model(inputs=latent_inputs, outputs=outputs, name="decoder")

# Define VAE model.
outputs = decoder(z)
vae = tf.keras.Model(inputs=original_inputs, outputs=outputs, name="vae")

In [ ]:
# Add KL divergence regularization loss.
kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
reconstruction_loss = binary_crossentropy(original_inputs,outputs)
reconstruction_loss *= original_dim
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

In [ ]:
epoch_num=200

# Create a callback that saves the model's weights
#https://jonathan-hui.medium.com/tensorflow-save-restore-model-75a1e6d3b9a6
#https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint
callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
        filepath="mymodel_{epoch}", save_weights_only=True,
        save_best_only = True, monitor = "val_loss", verbose =1,)]

In [ ]:
# Train.
optimizer = tf.keras.optimizers.Adam(learning_rate=.001)
vae.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
history = vae.fit(x_train, x_train, epochs=epoch_num, batch_size=128, callbacks=callbacks, validation_data=(x_validation, x_validation), verbose=1)

Epoch 1/200
166/166 [==============================] - 6s 13ms/step - loss: 594.6901 - val_loss: 417.1283

Epoch 00001: val_loss improved from inf to 417.12830, saving model to mymodel_1
Epoch 2/200
166/166 [==============================] - 2s 11ms/step - loss: 382.1248 - val_loss: 347.5745

Epoch 00002: val_loss improved from 417.12830 to 347.57452, saving model to mymodel_2
Epoch 3/200
166/166 [==============================] - 2s 11ms/step - loss: 330.7665 - val_loss: 308.6709

Epoch 00003: val_loss improved from 347.57452 to 308.67087, saving model to mymodel_3
Epoch 4/200
166/166 [==============================] - 2s 11ms/step - loss: 298.6427 - val_loss: 281.3226

Epoch 00004: val_loss improved from 308.67087 to 281.32260, saving model to mymodel_4
Epoch 5/200
166/166 [==============================] - 2s 11ms/step - loss: 275.1879 - val_loss: 260.3222

Epoch 00005: val_loss improved from 281.32260 to 260.32224, saving model to mymodel_5
Epoch 6/200
166/166 [====================

In [ ]:
x_train_encoded = encoder.predict(one_hot_encoded_seqs_array_train)
print(x_train_encoded)
print(x_train_encoded.shape)

[[ 3.305168   -3.230823   -1.280524   ... -2.8983583   0.778131
  -1.9180863 ]
 [-2.894323    3.8458076   1.5829015  ... -1.1644526   1.8173697
  -1.9834888 ]
 [-2.438271   -2.9630668  -2.3478312  ...  0.6692424  -2.2854228
  -0.11958335]
 ...
 [ 0.6204716   0.97091436 -0.5678681  ...  4.1817408   3.446762
   1.9125327 ]
 [-0.83783567 -1.5896701   2.1610584  ... -3.1563222  -3.052564
  -0.92010325]
 [-0.93810326 -4.277172   -2.233204   ...  3.0352535  -2.7051048
   2.6935296 ]]
(12906, 50)


In [ ]:
species_unique_labels = list(set(label_seq))
print(species_unique_labels)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [ ]:
#Calculate mean and inverse covariance matrix per species
species_unique_labels = list(set(label_seq))
label_array = np.array(label_seq)
label_df = pd.DataFrame(label_array)
x_train_encoded_df = pd.DataFrame(x_train_encoded)
error = np.eye(latent_dim)*0.01

mean_per_species = [[] for x in range(len(species_unique_labels))]
inverse_covariance_per_species = [[] for x in range(len(species_unique_labels))]
for i in range(len(species_unique_labels)):
    Index_of_labels = np.where(label_df == species_unique_labels[i])
    seqs_embedding_by_species = x_train_encoded_df.iloc[Index_of_labels[0],:]
    seqs_embedding_by_species_transpose = np.transpose(seqs_embedding_by_species)
    mean_by_species = np.mean(seqs_embedding_by_species, axis = 0)
    mean_per_species[i].append(np.array(mean_by_species))
    covariance_by_species = np.cov(seqs_embedding_by_species_transpose) + error
    cov_inv_by_species = np.linalg.inv(covariance_by_species)
    inverse_covariance_per_species[i].append(cov_inv_by_species)

print(mean_per_species)
print(np.array(mean_per_species).shape)
print(inverse_covariance_per_species)
print(np.array(inverse_covariance_per_species).shape)

Streaming output truncated to the last 5000 lines.
       ...,
       [-5.17656059e+00, -1.43328146e+00,  4.49979484e+00, ...,
         8.91704337e+01, -6.10459000e+00,  5.87795434e+00],
       [ 9.28662491e-01, -1.69819421e-02,  4.51829458e+00, ...,
        -6.10459000e+00,  9.13181520e+01, -1.07272464e+00],
       [ 5.47562888e+00,  3.20616245e+00, -1.66347080e+00, ...,
         5.87795434e+00, -1.07272464e+00,  8.58475777e+01]])], [array([[ 7.89482818e+01,  4.06496106e+00,  3.65644871e-02, ...,
        -2.18053011e+00,  5.99066789e-01, -2.90667588e+00],
       [ 4.06496106e+00,  7.00924472e+01, -4.06508637e-01, ...,
        -3.66866243e+00, -3.14306909e+00,  1.62909603e+00],
       [ 3.65644871e-02, -4.06508637e-01,  8.17727003e+01, ...,
         3.08671813e+00,  2.36318516e+00, -3.26699572e+00],
       ...,
       [-2.18053011e+00, -3.66866243e+00,  3.08671813e+00, ...,
         8.41162993e+01, -3.93042293e+00,  3.05547932e+00],
       [ 5.99066789e-01, -3.14306909e+00,  2.36318516

In [ ]:
mean_per_species = np.array(mean_per_species).reshape(len(mean_per_species), latent_dim)
print(np.array(mean_per_species).shape)
inverse_covariance_per_species = np.array(inverse_covariance_per_species).reshape(len(inverse_covariance_per_species), latent_dim, latent_dim)
print(np.array(inverse_covariance_per_species).shape)

(1202, 50)
(1202, 50, 50)


In [ ]:
#Get test embeddings
x_test_encoded = encoder.predict(one_hot_encoded_seqs_array_test)
print(x_test_encoded)
print(np.array(x_test_encoded).shape)

[[ 6.8121586  -0.33882752  2.0612073  ... -0.6408464  -2.0597758
  -3.822011  ]
 [ 0.41968217 -3.541583   -3.0972197  ...  0.28772417  1.666467
  -5.1380095 ]
 [-2.3164315   2.3064868  -0.21223113 ... -0.45061973 -1.6836551
  -1.1180353 ]
 ...
 [-2.1581888   0.17326629  0.47913617 ... -0.32919356 -2.1072106
   0.53075147]
 [ 1.4681729  -2.4537308   3.4954245  ...  1.5664029  -3.028554
   0.01798733]
 [ 4.552355    0.0617582  -2.7919874  ...  1.0637027  -1.8276128
  -4.445814  ]]
(8306, 50)


In [ ]:
#Calculate mahalanobis distance for each test instance

In [ ]:
mahalanobis_distance_test = np.zeros((len(x_test_encoded), len(species_unique_labels)))
for i in range (len(x_test_encoded)):
  for j in range(len(mean_per_species)):
    mahalanobis_distance = distance.mahalanobis(x_test_encoded[i], mean_per_species[j], inverse_covariance_per_species[j])
    mahalanobis_distance_test[i,j] = mahalanobis_distance

In [ ]:
print(mahalanobis_distance_test)
print(mahalanobis_distance_test.shape)

[[224.70096529 258.88165059 234.25956273 ... 197.91170081 159.8732887
  224.64051658]
 [250.20554665 274.93791915 224.15761014 ... 211.55463955 103.1978084
  202.00206527]
 [205.60684847 200.35661577 249.04444526 ... 189.62828101 169.62660848
  213.12019233]
 ...
 [175.35961824 208.82141884 187.5378257  ... 160.77670566  81.47505332
  149.35538813]
 [224.91323518 234.55857101 230.55084244 ... 188.95004682 143.10336939
  193.93321915]
 [268.00757385 292.18304795 238.52022701 ... 251.54587772 201.97093126
  225.25085477]]
(8306, 1202)


In [ ]:
def get_seq_ids(filename):
    df = pd.read_csv(filename)
    ids = df['id']
    
    return np.array(ids)

filename_features_test = 'test_features.csv'
test_ids = get_seq_ids(filename_features_test)
print(test_ids)

frames = [pd.DataFrame(test_ids), pd.DataFrame(mahalanobis_distance_test)]
output_data= np.concatenate(frames, axis=1)
output_df = pd.DataFrame(output_data)
output_df.to_csv('mahalanobis_distance_from_vae_embedding.csv', index=False)

[   1    2    3 ... 8304 8305 8306]


In [ ]:
files.download("mahalanobis_distance_from_vae_embedding.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
mahalanobis_distance_df = pd.DataFrame(mahalanobis_distance_test)

a = mahalanobis_distance_df.values
a.sort(axis=1)  # no ascending argument
a = a[:, ::-1]  # so reverse
lowest_distances = a[:,1201]
print(lowest_distances)

[25.65142299 11.95867659  3.14541466 ... 48.28986849  2.63547376
  4.08807669]


In [ ]:
frames = [pd.DataFrame(test_ids), pd.DataFrame(np.array(lowest_distances).reshape(len(lowest_distances),1))]
output_data= np.concatenate(frames, axis=1)
output_df = pd.DataFrame(output_data)
output_df.to_csv('lowest_mahalanobis_distance_from_vae_embedding.csv', index=False)

In [ ]:
files.download('lowest_mahalanobis_distance_from_vae_embedding.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
labels = train_labels_df['labels']
print(max(labels))
label_array = np.array(labels)
print(Counter(label_array))

1213
Counter({1162: 137, 625: 116, 1167: 78, 467: 78, 510: 74, 406: 67, 404: 64, 507: 61, 891: 57, 1212: 56, 1193: 54, 8: 50, 606: 48, 423: 44, 1177: 41, 688: 40, 735: 39, 878: 39, 352: 38, 407: 37, 327: 36, 1173: 35, 870: 34, 473: 32, 1037: 31, 604: 31, 511: 30, 1191: 30, 514: 29, 841: 29, 509: 29, 525: 29, 542: 29, 9: 28, 34: 27, 562: 27, 925: 27, 894: 27, 992: 26, 255: 26, 991: 26, 741: 26, 796: 25, 1184: 25, 717: 25, 386: 24, 516: 24, 381: 24, 140: 23, 417: 23, 560: 22, 5: 22, 205: 22, 324: 22, 1028: 22, 1171: 22, 457: 21, 203: 21, 1160: 21, 881: 21, 759: 21, 563: 21, 115: 21, 1063: 21, 676: 21, 716: 21, 883: 20, 387: 20, 373: 20, 310: 20, 177: 20, 1067: 19, 512: 19, 225: 19, 97: 19, 172: 19, 928: 19, 943: 19, 1146: 19, 26: 19, 1059: 19, 1096: 19, 468: 18, 1188: 18, 686: 18, 440: 18, 1027: 18, 301: 18, 758: 18, 285: 18, 1035: 18, 708: 18, 118: 18, 587: 18, 471: 18, 1175: 17, 250: 17, 465: 17, 892: 17, 1062: 17, 1136: 17, 93: 17, 501: 17, 1144: 17, 348: 17, 771: 17, 1133: 17, 620: 1

In [ ]:
#Finding outliers based on the threshold.
outlier = np.ones(len(x_test_encoded))
threshold = 100
for i in range (len(x_test_encoded)):
    if lowest_distances[i] < threshold:
      outlier[i] = 0

print(sum(outlier))

102.0


In [ ]:
frames = [pd.DataFrame(test_ids), pd.DataFrame(np.array(outlier).reshape(len(outlier),1))]
output_data= np.concatenate(frames, axis=1)
output_df = pd.DataFrame(output_data)
output_df.to_csv('outliers_threshold_100_mahalanobis_distance_from_vae_embedding.csv', index=False, header=["id","labels"])

In [ ]:
files.download('outliers_threshold_100_mahalanobis_distance_from_vae_embedding.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>